In [2]:
import mysql.connector
import pandas as pd

# Database connection

### Functions to run queries

In [3]:
def connect():
    con = mysql.connector.connect(
        host="35.193.209.4",
        user="0213663",
        password="0213663",
        database="up_0213663"
    )
    cur = con.cursor()
    return con, cur

def close(con, cur):
    cur.close()
    con.close()
    
def run(query):
    con, cur = connect()
    cur.execute(query)
    close(con, cur)
    
def retrieve(query):
    con, cur = connect()
    cur.execute(query)
    rows = cur.fetchall()
    columns = [d[0] for d in cur.description]
    close(con, cur)
    return pd.DataFrame(rows, columns=columns)

## Tables

![DB](DB.png)

### Query to create tables

```sql
CREATE TABLE `students` (
  `id` CHAR(10) PRIMARY KEY NOT NULL,
  `name` VARCHAR(128) NOT NULL,
  `last_names` VARCHAR(128) NOT NULL,
  `alias` VARCHAR(128)
);

CREATE TABLE `classes` (
  `class_number` INT AUTO_INCREMENT PRIMARY KEY NOT NULL,
  `topic` VARCHAR(128) NOT NULL,
  `created_at` TIMESTAMP NOT NULL DEFAULT CURRENT_TIME
);

CREATE TABLE `questions` (
  `class_number` INT PRIMARY KEY NOT NULL,
  `question` VARCHAR(256),
);

CREATE TABLE `answers` (
  `class_number` INT PRIMARY KEY NOT NULL,
  `student_id` CHAR(10) NOT NULL,
  `given_answer` VARCHAR(256) NOT NULL
);

CREATE TABLE `attendance` (
  `class` INT PRIMARY KEY NOT NULL,
  `student_id` VARCHAR(256) NOT NULL,
  `attendance` bool NOT NULL DEFAULT 0
);
```

### Query to add relationships

```sql
ALTER TABLE `classes` ADD FOREIGN KEY (`student_id`) REFERENCES `students` (`id`);

ALTER TABLE `questions` ADD FOREIGN KEY (`class_number`) REFERENCES `classes` (`class_number`);

ALTER TABLE `questions` ADD FOREIGN KEY (`class_number`) REFERENCES `answers` (`class_number`);

ALTER TABLE `students` ADD FOREIGN KEY (`id`) REFERENCES `answers` (`student_id`);

ALTER TABLE `classes` ADD FOREIGN KEY (`class_number`) REFERENCES `attendance` (`class_number`);

ALTER TABLE `students` ADD FOREIGN KEY (`id`) REFERENCES `attendance` (`student_id`);
```

### Run the queries

In [5]:
with open('Tables.sql', 'r') as file:
    sql_string = file.read()
query = sql_string.replace('\n', '').replace('  ', ' ').replace(';', '; ')
run(query)

Check existance

In [5]:
retrieve('SHOW TABLES')

,Tables_in_up_0213663
0,answers
1,attendance
2,classes
3,questions
4,students


### Inserts per table (examples)

**students**
```sql
INSERT INTO students (id, name, last_names, alias) VALUES ('0224604', 'Paulina', 'Barba Mendoza', 'Pau'), ('0213663', 'Eduardo', 'Solano Jaime', 'Eduardo');
```
**classes**
```sql
INSERT INTO classes (class, question, topic) VALUES (1, 'Favorite food', 'Django'), (2, 'Favourite Number', 'Cloud Computing')
```
**questions**
```sql
INSERT INTO questions (class, question) VALUES (1, 'Favorite food');
```
**answers**
```sql
INSERT INTO answers (class, id, given_answer) VALUES (1, '0224604', 'Sushi'), (1, '0213663', 'Tacos');
```
**attendance**
```sql
INSERT INTO attendance (class, id, attendance) VALUES (1, '0224604', 1), (1, '0213663');
```